In [1]:
import pandas as pd
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARMA
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
%matplotlib inline
#%matplotlib notebook
sns.set(rc={'figure.figsize':(11.7,8.27)})
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime,timedelta
from itertools import product
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

In [2]:
df = pd.read_pickle('unpiv_df_f1.pkl')

In [6]:
# filte keys which are available in 2019-2020
df2 = df.loc[df.index <= pd.to_datetime('2019-06-01')]

#df3 = df2.groupby('key')['siv_demand'].apply(lambda x: (x==0).sum()).reset_index(name='count')
df3 = df2.query('siv_demand ==0').groupby('key').size().reset_index(name='count_zeros')
#df3 = df2.groupby('key')['siv_demand'].apply(lambda x: (x==0).count()).reset_index(name='count_demand')# not working
#df3.sort_values('key',ascending=False)

# df3.sample(20)
# df3.count_zeros.value_counts().sort_values()
keys_model = df3.query('count_zeros <= 6').key.tolist()
# len(keys_model)
# model_df = df2.loc[df2.key.isin(keys_model)]

In [52]:
model_df.shape

(3906, 4)

In [64]:
# fixed params
base_time_frame = pd.date_range(start="2016-01-01",end="2020-02-01",freq="MS")
train_data_max_date = datetime.strptime('20190701',"%Y%m%d")
forecast_range = list(filter(lambda x: x > train_data_max_date,base_time_frame))
res = list()

In [114]:
res = list()
for key in keys_model:
    tm_df = df.query('key == @key')['siv_demand'].sort_index()
    #ser = ser[ser<=ser.quantile(0.99)]
    #ser = ser[ser < ser.quantile(0.75)+ (ser.quantile(0.75)-ser.quantile(0.25))*2]
    #tm_df = pd.DataFrame(index=base_time_frame)
    #tm_df = pd.concat([tm_df,ser],axis=1)
    sample_data = tm_df.copy()
    sample_data.loc[sample_data==0] = np.nan
    sample_data = sample_data.interpolate(method='time')
    tm_df.fillna(0,inplace=True)
    print(key)
    
    for tm in forecast_range:
        train_data = sample_data.loc[:tm][:-2]  # setting train data 2 months prior to forecast month
        demand_act = tm_df.loc[tm]
        naive_fcst = train_data.iloc[-1]
        avg_fcst =  train_data.mean()
        sma2 = train_data.rolling(2).mean().iloc[-1]
        sma5 = train_data.rolling(5).mean().iloc[-1]
        fit2 = SimpleExpSmoothing(np.asarray(train_data)).fit(smoothing_level=0.6,optimized=False)
        ses = fit2.forecast(2)[1]
        fit1 = Holt(np.asarray(train_data)).fit(smoothing_level = 0.3,smoothing_slope = 0.1)
        holt = fit1.forecast(2)[1]
        hwm_model = ExponentialSmoothing(np.asarray(train_data) ,seasonal_periods=7 ,trend='add', seasonal
                ='add',).fit()
        hwm = hwm_model.forecast(2)[1]
        #print(tm.date())
        fcst = 0
        stderr = 0
        up_conf = 0
        low_conf = 0
        try:
            model = ARIMA(train_data, order=(4,1,2))
            model_fit = model.fit(transparams=False,disp=-1,trend='c',maxiter = 5000)#start_params =5
            #print(model_fit.forecast())
            fcst,stderr,conf = model_fit.forecast(steps =2)
            fcst = 0 if np.isnan(round(fcst[1])) else round(fcst[1])
            stderr = 0 if np.isnan(round(stderr[1])) else round(stderr[1])
            low_conf = 0 if np.isnan(round(conf[1][0])) else round(conf[1][0])
            up_conf = 0 if np.isnan(round(conf[1][1])) else round(conf[1][1])
            #print(fcst,stderr)
        except Exception as e:
            print(e)
            fcst = 0
            fcst = 0
            stderr = 0
            up_conf = 0
            low_conf = 0
        
        res.append([key,tm,demand_act,fcst,stderr,low_conf,up_conf,naive_fcst,\
                    avg_fcst,sma2,sma5,ses,holt,hwm]) #tm.strftime("%Y-%m-%d")
res_df = pd.DataFrame(res,columns=['key','timestamp','actual','forecast','stdErr','lowPrInt','upPrInt','naive',\
                                  'avg_fcst','sma2','sma5','ses','holt','hwm'])

ANAND MOTORS-139284
Gulati Agencies Pvt. Ltd.-122310
Gulati Agencies Pvt. Ltd.-122467
Gulati Agencies Pvt. Ltd.-123336
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial AR coefficients are not stationary
You should induce stationarity, choose a different model order, or you can
pass your own start_params.
Gulati Agencies Pvt. Ltd.-130512
The computed initial MA coefficients are not inve

In [114]:
res = list()
for key in keys_model:
    ser = df.query('key == @key')['siv_demand'].sort_index()
    ser.fillna(0,inplace=True)
    ser1 = ser[ser<=ser.quantile(0.99)]
    #ser = ser[ser < ser.quantile(0.75)+ (ser.quantile(0.75)-ser.quantile(0.25))*2]
    tm_df = pd.DataFrame(index=base_time_frame)
    tm_df = pd.concat([tm_df,ser1],axis=1)
    sample_data = tm_df.copy()
    sample_data.loc[sample_data==0] = np.nan
    sample_data = sample_data.interpolate(method='time')
    tm_df.fillna(0,inplace=True)
    print(key)
    
    for tm in forecast_range:
        train_data = sample_data.loc[:tm][:-2]  # setting train data 2 months prior to forecast month
        demand_act = ser.loc[tm]
        naive_fcst = train_data.iloc[-1]
        avg_fcst =  train_data.mean()
        sma2 = train_data.rolling(2).mean().iloc[-1]
        sma5 = train_data.rolling(5).mean().iloc[-1]
        fit2 = SimpleExpSmoothing(np.asarray(train_data)).fit(smoothing_level=0.6,optimized=False)
        ses = fit2.forecast(2)[1]
        fit1 = Holt(np.asarray(train_data)).fit(smoothing_level = 0.3,smoothing_slope = 0.1)
        holt = fit1.forecast(2)[1]
        hwm_model = ExponentialSmoothing(np.asarray(train_data) ,seasonal_periods=7 ,trend='add', seasonal
                ='add',).fit()
        hwm = hwm_model.forecast(2)[1]
        #print(tm.date())
        fcst = 0
        stderr = 0
        up_conf = 0
        low_conf = 0
        try:
            model = ARIMA(train_data, order=(4,1,2))
            model_fit = model.fit(transparams=False,disp=-1,trend='c',maxiter = 5000)#start_params =5
            #print(model_fit.forecast())
            fcst,stderr,conf = model_fit.forecast(steps =2)
            fcst = 0 if np.isnan(round(fcst[1])) else round(fcst[1])
            stderr = 0 if np.isnan(round(stderr[1])) else round(stderr[1])
            low_conf = 0 if np.isnan(round(conf[1][0])) else round(conf[1][0])
            up_conf = 0 if np.isnan(round(conf[1][1])) else round(conf[1][1])
            #print(fcst,stderr)
        except Exception as e:
            print(e)
            fcst = 0
            fcst = 0
            stderr = 0
            up_conf = 0
            low_conf = 0
        
        res.append([key,tm,demand_act,fcst,stderr,low_conf,up_conf,naive_fcst,\
                    avg_fcst,sma2,sma5,ses,holt,hwm]) #tm.strftime("%Y-%m-%d")
res_df = pd.DataFrame(res,columns=['key','timestamp','actual','forecast','stdErr','lowPrInt','upPrInt','naive',\
                                  'avg_fcst','sma2','sma5','ses','holt','hwm'])

ANAND MOTORS-139284
Gulati Agencies Pvt. Ltd.-122310
Gulati Agencies Pvt. Ltd.-122467
Gulati Agencies Pvt. Ltd.-123336
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial AR coefficients are not stationary
You should induce stationarity, choose a different model order, or you can
pass your own start_params.
Gulati Agencies Pvt. Ltd.-130512
The computed initial MA coefficients are not inve

In [56]:
# tm_df = df.query('key == @keys_model[0]')['siv_demand'].sort_index()

# tm_df.loc[tm_df==0] = np.nan

# sample_data = tm_df.interpolate(method='time')

# train_data = sample_data.loc[:forecast_range[0]][:-2]

# fit2 = SimpleExpSmoothing(np.asarray(train_data)).fit(smoothing_level=0.6,optimized=False)

# fit1 = Holt(np.asarray(train_data)).fit(smoothing_level = 0.3,smoothing_slope = 0.1)

# fit1.forecast(2)[1]

# model = ARIMA(train_data, order=(4,1,2))
# model_fit = model.fit(transparams=False,disp=-1,trend='c',maxiter = 5000)#start_params =5
# #print(model_fit.forecast())
# fcst,stderr,conf = model_fit.forecast(steps =2)

In [122]:
df_19['key'] = df_19.Distributor.astype(str)+"-"+df_19.Material.astype(str)

df_19.columns = ['Brand Architecture', 'Distributor', 'Distributor_AR', 'Material', \
                'timestamp', 'Package Group', 'Product Family', \
                'Product Line', 'Synthetic Indicator', 'Month', 'EMAPS_Forecast', \
                'ICB', 'SIV', 'SOV', 'key']

res_df_2 = pd.merge(res_df,df_19[['key','EMAPS_Forecast','timestamp']],how='left',on=['key','timestamp'])

model_cols =  ['forecast','naive', 'avg_fcst', 'sma2', 'sma5', 'ses', 'holt', 'hwm','EMAPS_Forecast']
error_ser = res_df_2.loc[:,model_cols].apply(lambda x: x - res_df_2.actual)


In [150]:
res_df_2.forecast.quantile([0.03,0.99])

0.03        0.00
0.99   60,891.00
Name: forecast, dtype: float64

In [152]:
res_df_2.loc[res_df_2.forecast >= 61000,'forecast'] = res_df_2.naive
res_df_2.loc[res_df_2.forecast < 0,'forecast'] = res_df_2.naive

In [153]:
mae_error = res_df_2.loc[:,model_cols].apply(lambda x: x - res_df_2.actual,axis=0).abs().mean(axis=0)

In [154]:
mae_error

forecast         3,844.70
naive            4,264.11
avg_fcst         3,443.37
sma2             3,807.32
sma5             3,436.11
ses              3,758.32
holt             3,514.81
hwm              3,636.85
EMAPS_Forecast   2,974.32
dtype: float64

In [190]:
res = list()
for key in keys_model:
    ser = df.query('key == @key')['siv_demand'].sort_index()
    if ser[0] == 0:
        ser[0] = ser[:5].mean()
    ser.fillna(0,inplace=True)
    ser1 = ser[ser<=ser.quantile(0.99)]
    #ser = ser[ser < ser.quantile(0.75)+ (ser.quantile(0.75)-ser.quantile(0.25))*2]
    tm_df = pd.DataFrame(index=base_time_frame)
    tm_df = pd.concat([tm_df,ser1],axis=1)
    sample_data = tm_df.copy()
    sample_data[sample_data==0] = np.nan
    sample_data = sample_data.interpolate(method='time')
    tm_df.fillna(0,inplace=True)
    print(key)
    
    for tm in forecast_range:
        train_data = sample_data.loc[:tm][:-2]  # setting train data 2 months prior to forecast month
        demand_act = ser.loc[tm]

        ses_model = SimpleExpSmoothing(np.asarray(train_data)).fit(smoothing_level=0.1,optimized=False)
        ses1 = ses_model.forecast(2)[1]
#         ses_model = SimpleExpSmoothing(np.asarray(train_data)).fit(smoothing_level=0.1,optimized=True)
#         ses2 = ses_model.forecast(2)[1]
#         ses_model = SimpleExpSmoothing(np.asarray(train_data)).fit(smoothing_level=0.05,optimized=True)
#         ses3 = ses_model.forecast(2)[1]
#         ses_model = SimpleExpSmoothing(np.asarray(train_data)).fit(smoothing_level=0.1,optimized=False)
#         ses3 = ses_model.forecast(2)[1]
        
        holt_model = Holt(np.asarray(train_data)).fit(smoothing_level = 0.1,smoothing_slope = 0.1)
        holt1 = holt_model.forecast(2)[1]
#         holt_model = Holt(np.asarray(train_data)).fit(smoothing_level = 0.2,smoothing_slope = 0.15)
#         holt2 = holt_model.forecast(2)[1]
#         holt_model = Holt(np.asarray(train_data)).fit(smoothing_level = 0.05,smoothing_slope = 0.05)
#         holt3 = holt_model.forecast(2)[1]
#         holt_model = Holt(np.asarray(train_data)).fit(smoothing_level = 0.1,smoothing_slope = 0.05)
#         holt4 = holt_model.forecast(2)[1]
        
        
        hwm_model = ExponentialSmoothing(np.asarray(train_data) ,seasonal_periods=5 ,trend='add', seasonal
                ='add').fit()
        hwm1 = hwm_model.forecast(2)[1]
#         hwm_model = ExponentialSmoothing(np.asarray(train_data) ,seasonal_periods=7 ,trend='mul', seasonal
#                 ='mul').fit()
#         hwm2 = hwm_model.forecast(2)[1]
#         hwm_model = ExponentialSmoothing(np.asarray(train_data) ,seasonal_periods=5 ,trend='mul', seasonal
#                 ='add',damped=True).fit()
#         hwm3 = hwm_model.forecast(2)[1]
#         hwm_model = ExponentialSmoothing(np.asarray(train_data) ,seasonal_periods=5 ,trend='mul', seasonal
#                 ='add').fit()
#         hwm4 = hwm_model.forecast(2)[1]
        
        #print(tm.date())
        fcst = 0
        stderr = 0
        up_conf = 0
        low_conf = 0
        try:
            model = ARIMA(train_data, order=(2,1,1))
            model_fit = model.fit(transparams=False,disp=-1,trend='c',maxiter = 5000)#start_params =5
            #print(model_fit.forecast())
            fcst,stderr,conf = model_fit.forecast(steps =2)
            fcst = 0 if np.isnan(round(fcst[1])) else round(fcst[1])
            stderr = 0 if np.isnan(round(stderr[1])) else round(stderr[1])
            low_conf = 0 if np.isnan(round(conf[1][0])) else round(conf[1][0])
            up_conf = 0 if np.isnan(round(conf[1][1])) else round(conf[1][1])
            #print(fcst,stderr)
        except Exception as e:
            print(e)
            fcst = 0
            fcst = 0
            stderr = 0
            up_conf = 0
            low_conf = 0
        
        
        res.append([key,tm,demand_act,ses1,holt1,hwm1,fcst])
res_df_3 = pd.DataFrame(res,columns=['key','timestamp','actual','ses1','holt1','hwm1','arima'])

ANAND MOTORS-139284
Gulati Agencies Pvt. Ltd.-122310
Gulati Agencies Pvt. Ltd.-122467
Gulati Agencies Pvt. Ltd.-123336
Gulati Agencies Pvt. Ltd.-130512
Gulati Agencies Pvt. Ltd.-139362
Gulati Agencies Pvt. Ltd.-139432
Gulati Agencies Pvt. Ltd.-139433
Gulati Agencies Pvt. Ltd.-139434
Gulati Agencies Pvt. Ltd.-139461
Hans Lubricants & Auto Spares Private Limited-136071
Hans Lubricants & Auto Spares Private Limited-139470
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
Hans Lubricants & Auto Spares Private Limited-139471
Hans Lubricants & Auto Spares Private Limited-139474
Hans Lubricants & Auto Spares Private Limited-139487
M/S Ramesh Marketing-139358
M/S Ramesh Marketing-139362
MODI SALES AGENCY-139241
MODI SALES AGENCY-139242
MODI SALES AGENCY-139252
MODI SALES AGENCY-139261
MODI SALES AGENCY-139438
Mars Petrochem-122467
Mars Petrochem-139242
PON PURE CHEMICAL INDIA PRIVATE L

In [194]:
#res_df_4 = res_df_3.dropna()
res_df_3.loc[res_df_3.arima >= 61000,'arima'] = res_df_3.ses1
res_df_3.loc[res_df_3.arima < 0,'arima'] = res_df_3.ses1
res_df_4 = pd.merge(res_df_3,df_19[['key','EMAPS_Forecast','timestamp']],how='left',on=['key','timestamp'])

model_cols = ['ses1','holt1','hwm1','arima','EMAPS_Forecast']
mae_error = res_df_4.loc[:,model_cols].apply(lambda x: x - res_df_4.actual,axis=0).abs().mean(axis=0)
mae_error

ses1             3,269.97
holt1            3,329.31
hwm1             3,472.35
arima            3,409.79
EMAPS_Forecast   2,974.32
dtype: float64

In [195]:
res_df_4.to_excel('long_term_model_results_2.xlsx',index=False)